###PROBLEM: Do an link  prediction of graph
SUBMITTED BY: SWATHI ANANDRAM

####Initial Setup

#####Installing required packages

In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-geometric

import torch_geometric
print(torch_geometric.__version__)

1.12.1+cu113
     |████████████████████████████████| 7.9 MB 5.3 MB/s 
     |████████████████████████████████| 3.5 MB 4.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 467 kB 5.3 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.1.0.post1-py3-none-any.whl size=689859 sha256=109a32bbeea4bb7c916481f21c6e3e956bd8863c57ad45a48a22ae0406c56918
  Stored in directory: /root/.cache/pip/wheels/d1/cb/43/f7f2e472de4d7cff31bceddadc36d634e1e545fbc17961c282
Successfully built torch-geometric
2.1.0


In [2]:
import os.path as osp

import torch
from sklearn.metrics import roc_auc_score

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#####Loading Dataset

In [3]:
# load the Cora dataset
transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val=0.10, num_test=0.20, neg_sampling_ratio = 1.0,
                  is_undirected=True, add_negative_train_samples=False),
])
dataset = Planetoid('./data/Planetoid', name='Cora', transform=transform)
# After applying the `RandomLinkSplit` transform, the data is transformed from
# a data object to a list of tuples (train_data, val_data, test_data), with
# each element representing the corresponding split.
train_data, val_data, test_data = dataset[0]
print(train_data)
print(val_data)
print(test_data)

Processing...


Data(x=[2708, 1433], edge_index=[2, 7392], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[3696], edge_label_index=[2, 3696])
Data(x=[2708, 1433], edge_index=[2, 7392], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[1054], edge_label_index=[2, 1054])
Data(x=[2708, 1433], edge_index=[2, 8446], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[2110], edge_label_index=[2, 2110])


Done!


In [4]:
(train_data.edge_index[0] <= train_data.edge_index[1]).sum()

tensor(3696)

In [5]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        #return the indices of a non-zero element
        return (prob_adj > 0).nonzero(as_tuple=False).t()

In [6]:
model = Net(dataset.num_features, 128, 64).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

#### Model Training
Training with new negative samples for each training epoch

Validation/Test evaluation with roc_auc_score (Area Under the Receiver Operating Characteristic Curve)

In [7]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1))

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss


@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())



In [8]:
best_val_auc = final_test_auc = 0
for epoch in range(1, 101):
    loss = train()
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val = val_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_auc:.4f}')

print(f'Final Test: {final_test_auc:.4f}')

Epoch: 001, Loss: 0.6930, Val: 0.6771, Test: 0.7027
Epoch: 002, Loss: 0.6824, Val: 0.6733, Test: 0.6982
Epoch: 003, Loss: 0.7145, Val: 0.6800, Test: 0.6998
Epoch: 004, Loss: 0.6775, Val: 0.6960, Test: 0.7100
Epoch: 005, Loss: 0.6861, Val: 0.7539, Test: 0.7610
Epoch: 006, Loss: 0.6897, Val: 0.7794, Test: 0.7894
Epoch: 007, Loss: 0.6910, Val: 0.7331, Test: 0.7424
Epoch: 008, Loss: 0.6911, Val: 0.7056, Test: 0.7152
Epoch: 009, Loss: 0.6906, Val: 0.6991, Test: 0.7086
Epoch: 010, Loss: 0.6893, Val: 0.6988, Test: 0.7080
Epoch: 011, Loss: 0.6869, Val: 0.6990, Test: 0.7085
Epoch: 012, Loss: 0.6829, Val: 0.6979, Test: 0.7083
Epoch: 013, Loss: 0.6791, Val: 0.6946, Test: 0.7072
Epoch: 014, Loss: 0.6788, Val: 0.6910, Test: 0.7047
Epoch: 015, Loss: 0.6760, Val: 0.6879, Test: 0.6995
Epoch: 016, Loss: 0.6702, Val: 0.6849, Test: 0.6927
Epoch: 017, Loss: 0.6644, Val: 0.6838, Test: 0.6874
Epoch: 018, Loss: 0.6587, Val: 0.6859, Test: 0.6858
Epoch: 019, Loss: 0.6540, Val: 0.6927, Test: 0.6889
Epoch: 020, 

In [9]:
z = model.encode(test_data.x, test_data.edge_index)
final_edge_index = model.decode_all(z)

In [10]:
final_edge_index

tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [   0,    1,    2,  ..., 2704, 2706, 2707]])